In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,105.80,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.0,0.0,0.00,...,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.0
2,-0.00,160.41,-0.00,0.01,-0.00,0.00,-0.00,0.0,0.0,0.00,...,0.00,0.00,0.00,-0.00,0.00,0.01,0.00,0.00,0.00,0.0
3,-0.00,-0.00,59.42,-0.00,-0.00,0.00,0.00,-0.0,-0.0,0.00,...,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.0
4,-0.00,0.01,-0.00,150.84,0.00,0.00,0.00,0.0,0.0,0.00,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0
5,0.00,-0.00,-0.00,0.00,60.77,0.00,-0.00,0.0,-0.0,0.00,...,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.0
6,0.00,0.00,0.00,0.00,0.00,53.41,-0.00,-0.0,-0.0,-0.00,...,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.0
7,0.00,-0.00,0.00,0.00,-0.00,-0.00,75.95,0.0,-0.0,0.00,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.0
8,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,2.4,-0.0,0.00,...,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.0
9,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.0,21.4,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.0
10,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.0,0.0,53.72,...,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,-0.00002,-0.00004,-0.00003,0.00005,0.00004,0.00001,0.00000,0.00002,0.00002,...,-0.00001,-0.00002,0.00002,0.00000,0.00003,-0.00003,0.00002,0.00003,-0.00001,0.00000
2,-0.00002,1.00000,-0.00001,0.00004,-0.00002,0.00002,-0.00003,0.00000,0.00004,0.00002,...,0.00004,0.00003,0.00002,-0.00001,0.00005,0.00005,0.00006,0.00005,0.00002,0.00001
3,-0.00004,-0.00001,1.00000,-0.00002,-0.00000,0.00002,0.00003,-0.00001,-0.00001,0.00001,...,-0.00002,-0.00001,0.00005,0.00003,-0.00001,0.00004,-0.00003,-0.00001,-0.00001,-0.00003
4,-0.00003,0.00004,-0.00002,1.00000,0.00001,0.00004,0.00002,0.00001,0.00003,0.00001,...,-0.00002,0.00001,0.00000,-0.00004,0.00001,-0.00002,0.00002,-0.00001,0.00002,-0.00005
5,0.00005,-0.00002,-0.00000,0.00001,1.00000,0.00001,-0.00002,0.00000,-0.00000,0.00005,...,0.00002,0.00000,-0.00000,-0.00001,0.00003,0.00003,-0.00000,0.00002,-0.00002,-0.00004
6,0.00004,0.00002,0.00002,0.00004,0.00001,1.00000,-0.00003,-0.00002,-0.00011,-0.00002,...,-0.00003,0.00004,0.00001,0.00001,-0.00004,0.00005,-0.00001,-0.00002,0.00003,-0.00005
7,0.00001,-0.00003,0.00003,0.00002,-0.00002,-0.00003,1.00000,0.00002,-0.00000,0.00000,...,0.00002,-0.00001,0.00003,-0.00001,-0.00005,0.00003,0.00002,-0.00001,0.00001,-0.00007
8,0.00000,0.00000,-0.00001,0.00001,0.00000,-0.00002,0.00002,1.00000,-0.00000,0.00006,...,-0.00006,-0.00005,-0.00002,0.00002,0.00003,-0.00001,0.00001,-0.00001,-0.00001,0.00003
9,0.00002,0.00004,-0.00001,0.00003,-0.00000,-0.00011,-0.00000,-0.00000,1.00000,0.00003,...,0.00000,0.00001,0.00006,0.00004,0.00001,0.00005,-0.00005,-0.00001,-0.00004,-0.00001
10,0.00002,0.00002,0.00001,0.00001,0.00005,-0.00002,0.00000,0.00006,0.00003,1.00000,...,0.00001,-0.00001,-0.00004,0.00000,0.00003,-0.00004,-0.00004,-0.00001,-0.00003,0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.1544542891589122

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.20857897e+03 1.98374760e+03 8.84000853e+02 7.57762530e+02
 4.30710372e+02 3.80245907e+02 3.08123181e+02 2.91514541e+02
 2.66217184e+02 2.44031055e+02 2.26564917e+02 1.89812994e+02
 1.85238444e+02 1.72323970e+02 1.50624872e+02 1.44842293e+02
 1.35029835e+02 1.23983647e+02 1.17970025e+02 1.02908059e+02
 9.82684887e+01 9.58195868e+01 9.05216825e+01 8.63311381e+01
 7.99830650e+01 7.90175484e+01 7.69981889e+01 7.45096892e+01
 7.27641529e+01 6.84797295e+01 6.17796880e+01 5.90010077e+01
 5.66156717e+01 5.50615929e+01 5.26360615e+01 5.01727603e+01
 4.88977972e+01 4.85196088e+01 4.46318905e+01 4.12798240e+01
 4.03719054e+01 3.92670877e+01 3.57619371e+01 3.33752722e+01
 2.99963006e+01 2.69607473e+01 2.65928595e+01 2.34844070e+01
 2.14741713e+01 2.04132319e+01 1.70537170e+01 1.29475451e+01
 9.96402356e+00 7.88364138e+00 7.32134384e+00 6.76609696e+00
 6.06749926e+00 1.41570546e+00 2.78309642e-01 1.21758613e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,-0.004263,-0.007453,-0.002264,-0.006780,-0.002294,-0.002020,-0.002938,-0.000090,-0.000800,-0.002028,...,-0.000727,-0.001675,-0.003310,-0.001832,-0.001844,-0.004264,-0.000215,-0.001478,-0.002159,-0.000690
1,-0.006097,-0.010753,-0.003184,-0.009812,-0.003285,-0.002867,-0.004189,-0.000126,-0.001124,-0.002881,...,-0.001036,-0.002395,-0.004683,-0.002583,-0.002663,-0.006126,-0.000294,-0.002108,-0.003019,-0.000975
2,-0.008318,-0.019378,-0.003895,-0.016333,-0.004024,-0.003515,-0.005259,-0.000147,-0.001336,-0.003510,...,-0.001221,-0.002876,-0.006000,-0.003090,-0.003208,-0.008356,-0.000364,-0.002507,-0.003684,-0.001140
3,-0.011117,-0.029743,-0.005024,-0.024088,-0.005194,-0.004468,-0.006851,-0.000189,-0.001709,-0.004480,...,-0.001572,-0.003690,-0.007827,-0.003956,-0.004093,-0.011120,-0.000446,-0.003217,-0.004761,-0.001466
4,-0.014758,0.990290,-0.005369,-0.111430,-0.005551,-0.004664,-0.007686,-0.000186,-0.001682,-0.004698,...,-0.001542,-0.003765,-0.009074,-0.004099,-0.004228,-0.014587,-0.000430,-0.003238,-0.004991,-0.001461
5,-0.019693,0.100915,-0.006564,0.987213,-0.006747,-0.005646,-0.009464,-0.000221,-0.002014,-0.005707,...,-0.001873,-0.004565,-0.011448,-0.004985,-0.005147,-0.019666,-0.000520,-0.003943,-0.006068,-0.001771
6,-0.019794,0.027536,-0.005313,0.042191,-0.005507,-0.004553,-0.007999,-0.000178,-0.001586,-0.004574,...,-0.001474,-0.003618,-0.010020,-0.003947,-0.004157,-0.019685,-0.000410,-0.003116,-0.004842,-0.001345
7,-0.030852,0.032711,-0.007641,0.047777,-0.007894,-0.006526,-0.011658,-0.000242,-0.002245,-0.006603,...,-0.002076,-0.005184,-0.014572,-0.005645,-0.005933,-0.031111,-0.000568,-0.004415,-0.007012,-0.001928
8,-0.038934,0.026429,-0.008067,0.035304,-0.008323,-0.006833,-0.012659,-0.000261,-0.002318,-0.006942,...,-0.002148,-0.005411,-0.016231,-0.005920,-0.006176,-0.038927,-0.000596,-0.004642,-0.007463,-0.002018
9,-0.052263,0.022245,-0.008631,0.028521,-0.008957,-0.007362,-0.013834,-0.000261,-0.002402,-0.007350,...,-0.002231,-0.005691,-0.018228,-0.006259,-0.006582,-0.052420,-0.000624,-0.004864,-0.007919,-0.002077


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0042625641137712,
 1.0107534042950475,
 1.0038945776293224,
 1.024088492482793,
 1.005550737341665,
 1.0056461335366629,
 1.0079991906996244,
 1.0002415172572858,
 1.0023178487833615,
 1.0073495011470683,
 1.0098712541110801,
 1.000193367663476,
 1.012697260015091,
 1.0261794493870602,
 1.0118202953902857,
 1.007848471902252,
 1.0170157600543992,
 1.0044535686738858,
 0.9818218639346724,
 0.9962255933031355,
 0.999306689402856,
 1.0225451520452344,
 0.9881904228230796,
 0.8925640349025983,
 1.0001486238914266,
 0.9906935952948858,
 1.0038467600102208,
 0.989028816532813,
 1.0036352219510207,
 0.8119956346269949,
 0.9669671331116154,
 0.9860733323837501,
 1.0012053275456398,
 0.9786797774953715,
 0.9959840458039446,
 0.998576578195751,
 0.9996671008798022,
 1.0023467743336942,
 1.053268267580711,
 1.0113005646122333,
 1.0023934649813588,
 0.9768123212839096,
 1.013996456926223,
 1.0159102699027305,
 1.1060000713303282,
 0.9935081232630887,
 0.9794798273347681,
 1.0136270535665433,
 1